In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from os import listdir
from os.path import isfile, join

# some comment

embeddings = HuggingFaceEmbeddings(
    model_name = 'emilyalsentzer/Bio_ClinicalBERT'
)
 
text = "This is a test document."
query_result = embeddings.embed_query(text)


/Users/liangxin/anaconda3/envs/vector/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name emilyalsentzer/Bio_ClinicalBERT. Creating a new one with MEAN pooling.


In [2]:
#onlyfiles = [f for f in listdir('top_1000_txt') if isfile(join('top_1000_txt', f))]
onlyfiles=['note_211.txt']
raw_documents = []
for file in onlyfiles:
    print(file)
    raw_doc = TextLoader(f'top_1000_txt/{file}').load()
    raw_documents.extend(raw_doc)

text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=24)

note_211.txt


In [3]:
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, embeddings)

In [4]:
!pip install gpt4all

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [11]:
from gpt4all import GPT4All
model = GPT4All(model_name="mistral-7b-instruct-v0.1.Q4_0.gguf",
                             n_threads = 4,
                             allow_download=True)

Downloading: 100%|██████████| 4.11G/4.11G [04:31<00:00, 15.1MiB/s]
Verifying: 100%|██████████| 4.11G/4.11G [00:08<00:00, 489MiB/s]


In [4]:
from langchain_community.llms import GPT4All
llm = GPT4All(
            model="mistral-7b-instruct-v0.1.Q4_0.gguf",
            max_tokens=300,
            n_threads = 4, 
            temp=0.3,
            top_p=0.2,
            top_k=40,
            n_batch=8,
            seed=100,
            allow_download=True,
            verbose=True)

In [5]:

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [6]:
retriever = db.as_retriever(search_kwargs={"k": 1})

template = """You are an expert clinical assistant. You will receive a collection of clinical notes. You will summarize them in the style of a discharge summary.Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser())

In [7]:
chain.invoke('What allergies did the patient have?')

'Answer: The patient had no known allergies to drugs.'

In [8]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [9]:
from datasets import Dataset
questions= [
    "Has the patient got a history of diabetes?",
    "What allergies did the patient have?",
    "Does the patient use tobacco or alcohol?",
    "What medication is the patient taking?",
    "What measures were taken for the patient?",
    "What imaging studies were performed on the patient?",
    "What follow-up care was scheduled for the patient?",
    "What did C-spine show?",
    "Does the patient use alcohol?",
    "Does the patient use cigarettes or alcohol?",
    "Did patient use tobacco?"
]

ground_truths = [["No."],
                 ["The patient has no known allergies to drugs."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes) but denies alcohol use."],
                 ["Dilantin 100 IV TID."],
                 ["Intubation and placement of a left radial arterial line for close blood pressure monitoring with a goal of keeping systolic blood pressure under 140. head CT, C-spine, chest/abdomen/pelvis CT to assess injuries. The patient was started on Dilantin 100 IV TID. Monitoring in the Trauma Intensive Care Unit (TICU). Repeat head CT and gradual weaning. Discontinuation of the Foley catheter and intravenous lines.Transfer from the ICU to the general hospital floor.  MRI of the C-spine to evaluate for potential injuries, which showed no fractures or ligamentous injuries, leading to clearance of the C-spine and discontinuation of the C-collar.Evaluation by Neurosurgery and Physical Therapy, deeming the patient fit for discharge with instructions for home care and follow-up."],
                 ["The patient underwent head CT, C-spine, chest/abd/pelvis CT scans, and an MRI of the C-spine"],
                 ["An outpatient CT scan was scheduled to be done in 2 weeks, and a follow-up clinic appointment was scheduled in 2 weeks after the CT scan.No need to follow up in the trauma clinic but the patient may call the clinic with any questions."],
                 ["The MRI of the C-spine showed no fractures or ligamentous injuries."],
                 ["No, the patient denies alcohol use."],
                 ["The patient occasionally uses cigarettes and denies alcohol use."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes)."]

]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)

In [10]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [11]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 11
})

In [12]:
dataset.save_to_disk('mistral-7b-instruct-v0.1.Q4_0.gguf-dataset')

Saving the dataset (1/1 shards): 100%|██████████| 11/11 [00:00<00:00, 681.91 examples/s]


In [13]:
ragas_input_df = dataset.to_pandas()
display(ragas_input_df.head())

,question,answer,contexts,ground_truths
0,Has the patient got a history of diabetes?,"Answer: No, there is no mention of diabetes in...",[Patient recorded as having No Known Allergies...,[No.]
1,What allergies did the patient have?,Answer: The patient had no known allergies to ...,[Patient recorded as having No Known Allergies...,[The patient has no known allergies to drugs.]
2,Does the patient use tobacco or alcohol?,Answer: The patient uses occasional tobacco (c...,[Patient recorded as having No Known Allergies...,"[Yes, the patient occasionally uses tobacco (c..."
3,What medication is the patient taking?,Answer: The patient is not currently taking an...,[Patient recorded as having No Known Allergies...,[Dilantin 100 IV TID.]
4,What measures were taken for the patient?,Answer: The patient was intubated and had a le...,[Patient recorded as having No Known Allergies...,[Intubation and placement of a left radial art...


In [14]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)
     

In [15]:


def evaluate_ragas_dataset(dataset):
  result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
    
  )
  return result

In [16]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [17]:
qa_result = evaluate_ragas_dataset(dataset)

evaluating with [context_precision]


100%|██████████| 1/1 [00:01<00:00,  1.83s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:12<00:00, 12.12s/it]


evaluating with [answer_relevancy]


...

100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:13<00:00, 13.99s/it]


evaluating with [answer_similarity]


100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


In [18]:
print(qa_result)

{'context_precision': 0.2727, 'faithfulness': 0.8177, 'answer_relevancy': 0.9096, 'context_recall': 1.0000, 'context_relevancy': 0.1136, 'answer_correctness': 0.8605, 'answer_similarity': 1.0000}


In [19]:
df = qa_result.to_pandas()

In [20]:
df.to_csv("mistral-7b-instruct_ragas_result.csv")